In [12]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import TextLoader

loader = TextLoader("file.txt")
loader.load()



[Document(page_content='\n10 Academy Cohort A\nWeekly Challenge: Week 6\nPrecision RAG: Prompt Tuning For Building Enterprise Grade RAG Systems\n\nBusiness objective  \nPromptlyTech is an innovative e-business specializing in providing AI-driven solutions for optimizing the use of Language Models (LLMs) in various industries. The company aims to revolutionize how businesses interact with LLMs, making the technology more accessible, efficient, and effective. By addressing the challenges of prompt engineering, the company plays a pivotal role in enhancing decision-making, operational efficiency, and customer experience across various industries. PromptlyTech\'s solutions are designed to cater to the evolving needs of a digitally-driven business landscape, where speed and accuracy are key to staying competitive.\nThe company focuses on key services: Automatic Prompt Generation, Automatic Test Case Generation, and Prompt Testing and Ranking.\n1. Automatic Prompt Generation Service:\n    • 

In [13]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=80)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(texts, embeddings)

Created a chunk of size 1763, which is longer than the specified 1000
Created a chunk of size 1344, which is longer than the specified 1000
Created a chunk of size 3634, which is longer than the specified 1000
Created a chunk of size 1263, which is longer than the specified 1000


In [16]:
retriever = db.as_retriever(search_type="mmr")
docs = retriever.get_relevant_documents("Saturday 8pm ")
docs

[Document(page_content='10 Academy Cohort A\nWeekly Challenge: Week 6\nPrecision RAG: Prompt Tuning For Building Enterprise Grade RAG Systems', metadata={'source': 'file.txt'}),
 Document(page_content="Deliverables\nNOTE: Document should be a PDF stored in google drive or published blog link. DO NOT SUBMIT A LINK as PDF! If you want to submit pdf document, it should be the content of your report not a link. \nInterim Submission - Wednesday 8pm UTC\n    • Link to your code in GitHub\n        ◦ Repository where you will be using to complete the tasks in this week's challenge. A minimum requirement is that you have a well structured repository and some coding progress is made.\n\n\n    • A review report of your reading and understanding of Task 1 and any progress you made in other tasks. \nFeedback\nYou may not receive detailed comments on your interim submission, but will receive a grade.\nFinal Submission - Saturday 8pm UTC\n    • Link to your code in GitHub \n        ◦ Complete work  f

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Chroma

def data_retriever(file, query):
    # Load the document, split it into chunks, embed each chunk, and load it into the vector store.
    raw_documents = TextLoader(file).load()
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
    documents = text_splitter.split_documents(raw_documents)
    
    # Assuming OpenAIEmbeddings is properly initialized and available as `embeddings`
    db = Chroma.from_documents(documents, OpenAIEmbeddings())
    
    query = query
    docs = db.similarity_search(query)
    
    # Print each output individually
    for i in range(min(4, len(docs))):  # Avoid index out of range if there are less than 4 results
        print(docs[i].page_content)
    
    # Return None or any other value as needed
    return None


In [5]:
context = data_retriever('file.txt','brown')

Created a chunk of size 1004, which is longer than the specified 1000
Created a chunk of size 1203, which is longer than the specified 1000
Created a chunk of size 1025, which is longer than the specified 1000


Jessica was in charge of marketing at a Boston investment bank. This bank thought it understood startups, but over the next year, as she met friends of mine from the startup world, she was surprised how different reality was. And how colorful their stories were. So she decided to compile a book of interviews with startup founders.

When the bank had financial problems and she had to fire half her staff, she started looking for a new job. In early 2005 she interviewed for a marketing job at a Boston VC firm. It took them weeks to make up their minds, and during this time I started telling her about all the things that needed to be fixed about venture capital. They should make a larger number of smaller investments instead of a handful of giant ones, they should be funding younger, more technical founders instead of MBAs, they should let the founders remain as CEO, and so on.
Jessica was in charge of marketing at a Boston investment bank. This bank thought it understood startups, but ove

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

template = """Answer the question based only on the following context:

{context}

Question: brown
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()


def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke("What did the president say about technology?")


NameError: name 'retriever' is not defined